# ReAct Agents for LLMs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
"""
In this example we'll use LangChain to build an agent that can read Wikipedia articles, handle
context, parsing, and tool execution
"""

!pip install langchain-community langchain-core
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

In [3]:
# importing dependencies necessary

# for using OpenAI's GPT models
from langchain import OpenAI

# for allowing langchain to query Wikepedia articles
from langchain import Wikipedia

# for setting up na environment in which a ReAct agent can run autonomously
from langchain.agents import initialize_agent

# for defining tools to give a language models
from langchain.agents import Tool

# for defining the type of agent - there is not a lot of great documentation about agents in
# the LangChain docs. I think this sets the context used to inform the model how to behave
from langchain.agents import AgentType

# this abstracts text documents into a "search" and "lookup" function
from langchain.agents.react.base import DocstoreExplorer

In [4]:
# defining a docstore and telling the docstore to use LangChains hook for Wikipedia
docstore= DocstoreExplorer(Wikipedia())

# then we can search for an article
docstore.search('Dune (novel)')

<ipython-input-4-c37317a4882d>:2: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 1.0
  docstore= DocstoreExplorer(Wikipedia())


'Dune is a 1965 epic science fiction novel by American author Frank Herbert, originally published as two separate serials (1963–64 novel Dune World and 1965 novel Prophet of Dune) in Analog magazine. It tied with Roger Zelazny\'s This Immortal for the Hugo Award for Best Novel and won the inaugural Nebula Award for Best Novel in 1966. It is the first installment of the Dune Chronicles. It is one of the world\'s best-selling science fiction novels.\nDune is set in the distant future in a feudal interstellar society, descended from terrestrial humans, in which various noble houses control planetary fiefs.  It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or "spice", a drug that extends life and enhances mental abilities. Melange is also necessary for space navigation, which requires a kind of multidimensional awareness and fo

In [5]:
# looking up sections of the article which contain the word 'Lawrence'
docstore.lookup('Lawrence')

'(Result 1/2) After his novel The Dragon in the Sea was published in 1957, Herbert traveled to Florence, Oregon, at the north end of the Oregon Dunes. Here, the United States Department of Agriculture was attempting to use poverty grasses to stabilize the sand dunes. Herbert claimed in a letter to his literary agent, Lurton Blassingame, that the moving dunes could "swallow whole cities, lakes, rivers, highways." Herbert\'s article on the dunes, "They Stopped the Moving Sands", was never completed (and only published decades later in The Road to Dune), but its research sparked Herbert\'s interest in ecology and deserts.\nHerbert further drew inspiration from Native American mentors like "Indian Henry" (as Herbert referred to the man to his son; likely a Henry Martin of the Hoh tribe) and Howard Hansen. Both Martin and Hansen grew up on the Quileute reservation near Herbert\'s hometown. According to historian Daniel Immerwahr, Hansen regularly shared his writing with Herbert. "White men 

In [6]:
# searching for the article 'Dune'  which failes even though there is an article named 'Dune'
docstore.search('Dune')

"Could not find [Dune]. Similar: ['Dune', 'Dune: Part Two', 'List of Dune characters', 'Dune (novel)', 'Dune (2021 film)', 'Dune (franchise)', 'Dune Messiah', 'Dune (disambiguation)', 'Dune: Prophecy', 'Sandworm (Dune)']"

In [7]:
# searching for 'Sand Dune' results in the article 'Dune'
docstore.search('Sand Dune')

'A dune is a landform composed of wind- or water-driven sand. It typically takes the form of a mound, ridge, or hill. An area with dunes is called a dune system or a dune complex.  A large dune complex is called a dune field, while broad, flat regions covered with wind-swept sand or dunes, with little or no vegetation, are called ergs or sand seas. Dunes occur in different shapes and sizes, but most kinds of dunes are longer on the stoss (upflow) side, where the sand is pushed up the dune, and have a shorter slip face in the lee side. The valley or trough between dunes is called a dune slack.\nDunes are most common in desert environments, where the lack of moisture hinders the growth of vegetation that would otherwise interfere with the development of dunes. However, sand deposits are not restricted to deserts, and dunes are also found along sea shores, along streams in semiarid climates, in areas of glacial outwash, and in other areas where poorly cemented sandstone bedrock disintegra

In [8]:
"""
We have a docstore where we can search for documents and lookup sections within those documents
based on a keyword or phrase. Now we'll expose these as LangChain "Tools"
"""

tools= [
    Tool(
        name='Search',
        func=docstore.search,
        description='useful for when you need to ask with search'
    ),
    Tool(
        name='Lookup',
        func=docstore.lookup,
        description='useful for when you need to ask with lookup'
    )
]

# Defining a ReAct Agent

In [10]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
client= OpenAI()

"""
The language model understands our tools because, when initializing an agent, a context is
automatically created which shows the model several examples of tool usage
"""

llm= OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct')
react_agent= initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

<ipython-input-10-67a367ebd245>:8: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  react_agent= initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)


In [11]:
"""
The context allows the language model to understand the general structure of ReAct, and will
entice it to follow the "Thought", "Action", and "Observation" format
"""

react_agent.agent.llm_chain.prompt.template

'\n\nQuestion: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It does not mention the eastern sector. So I need to look up eastern sector.\nAction: Lookup[eastern sector]\nObservation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.\nThought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.\nAction: Search[High Plains]\nObservation: High Plains refers to one of two distinct land regions\nThought: I need to instead search High Plains (United States).\nAction: Search[High Pla

# Executing our ReAct Agent

In [ ]:
prompt= 'What is the age of the president of the United States? The current date is Sep 21 2024.'
react_agent.run(prompt)

# ReAct in Python from Scratch

In [ ]:
"""
LangChain does have a nice wrapper around BingSearch, so we'll be using LangChain to help us with
that, but we'll do all the ReAct stuff ourselves
"""

# testing out bing search
os.environ["BING_SUBSCRIPTION_KEY"]= userdata.get('BING_SUBSCRIPTION_KEY')
os.environ["BING_SEARCH_URL"]= "https://api.bing.microsoft.com/v7.0/search"

from langchain.utilities import BingSearchAPIWrapper

def BSearch(input):
    # expects an arbitrary text
    search= BingSearchAPIWrapper()

    return (search.run(input))


BSearch('How many regions are there in Italy?')

In [ ]:
# https://towardsdatascience.com/llm-agents-intuitively-and-exhaustively-explained-8905858e18e2